In [1]:
import os
import time 
import folium
import collections
import numpy as np
import pandas as pd
import networkx as nx
from folium.map import *
from folium import plugins
from folium.plugins import FloatImage
from tqdm import tnrange, tqdm_notebook
from folium.plugins import MeasureControl
from fibonacci_heap import Fibonacci_heap

In [2]:
files_names = os.listdir('./Files')

In [3]:
files_names

['USA-road-d.CAL.co', 'USA-road-d.CAL.gr', 'USA-road-t.CAL.gr']

In [4]:
os.getcwd() + '\\Files\\' + files_names[0]

'C:\\Users\\Giorgio\\Documents\\Data Science\\ADM\\HW5\\Files\\USA-road-d.CAL.co'

In [5]:
dataframes_names = ['coordinates', 'physical_dist', 'time_dist']

In [6]:
for k in tnrange(3):
    if os.getcwd() + '\\Files\\' + files_names[k] == os.getcwd() + '\\Files\\' + files_names[0]:
        globals()[dataframes_names[k]] = pd.read_csv(os.getcwd() + '\\Files\\' + files_names[0], skiprows = 7, sep = " ", 
                                          delimiter = " ", names = ["Character", "ID_Node", "Longitude", "Latitude"],
                                          index_col = None, usecols = None, encoding = 'ISO-8859-1')
        eval(dataframes_names[k]).drop(columns = ["Character"], inplace = True)
    elif os.getcwd() + '\\Files\\' + files_names[k] == os.getcwd() + '\\Files\\' + files_names[1]:
        globals()[dataframes_names[k]] = pd.read_csv(os.getcwd() + '\\Files\\' + files_names[1], skiprows = 7, sep = " ", 
                                          delimiter = " ", names = ["Character", "Node_1", "Node_2", "Physical_distance"],
                                          index_col = None, usecols = None, encoding = 'ISO-8859-1')
        eval(dataframes_names[k]).drop(columns = ["Character"], inplace = True)
    elif os.getcwd() + '\\Files\\' + files_names[k] == os.getcwd() + '\\Files\\' + files_names[2]:
        globals()[dataframes_names[k]] = pd.read_csv(os.getcwd() + '\\Files\\' + files_names[2], skiprows = 7, sep = " ", 
                                          delimiter = " ", names = ["Character", "Node_1", "Node_2", "Time_distance"],
                                          index_col = None, usecols = None, encoding = 'ISO-8859-1')
        eval(dataframes_names[k]).drop(columns = ["Character"], inplace = True)

In [69]:
coordinates.head()

,ID_Node,Longitude,Latitude
0,1,-114315309,34133550
1,2,-114223946,34176221
2,3,-114307299,34148791
3,4,-114318765,34138889
4,5,-114347300,34042614


In [70]:
physical_dist.head()

,Node_1,Node_2,Physical_distance
0,1,1048577,456
1,1048577,1,456
2,2,1048578,2389
3,1048578,2,2389
4,3,1048579,358


In [71]:
time_dist.head()

,Node_1,Node_2,Time_distance
0,1,1048577,1139
1,1048577,1,1139
2,2,1048578,5972
3,1048578,2,5972
4,3,1048579,895


In [7]:
complete = pd.merge(physical_dist, time_dist, on = ['Node_1', 'Node_2'])

In [72]:
complete.head()

,Node_1,Node_2,Physical_distance,Time_distance
0,1,1048577,456,1139
1,1048577,1,456,1139
2,2,1048578,2389,5972
3,1048578,2,2389,5972
4,3,1048579,358,895


In [8]:
G = nx.from_pandas_edgelist(complete, 'Node_1', 'Node_2', ['Physical_distance', 'Time_distance'], create_using = nx.DiGraph())

In [73]:
nx.is_directed(G)

True

In [61]:
# Custom function to run a BFS over the graph and checking it a node is connected or not
def b_f_s(graph, root): 
    visited, queue = set(), collections.deque([root])
    while queue: 
        vertex = queue.popleft()
        for neighbour in graph[vertex]: 
            if neighbour not in visited: 
                visited.add(neighbour) 
                queue.append(neighbour)
    return visited

# Custom function created to get the distance attribute from each edge.
def get_weight(graph, node_a, node_b, measure):
    if measure == 'network':
        return 1
    elif measure == 'time':
        return graph.get_edge_data(node_a, node_b)['Time_distance']
    elif measure == 'physical':
        return graph.get_edge_data(node_a, node_b)['Physical_distance']

# Custom function to implement the Dijkstra algorithm
def dijkstra(graph, source, destination, measure = 'network'):
    # The first if condition is deployed to avoid the case of an erroneous distance measure typing.
    if measure in ['network', 'time', 'physical']:
        # Here we decided to prevent the algorithm from starting by checking whether a node is not connected
        # via BFS.
        if destination in b_f_s(graph, source):
            # The variable 'shortest paths' is basically a dictionary where the keys are nodes and the values are a tuple
            # containing the couple (previous node, weight). We initialize it with the source vertex and set its weight to 0.
            shortest_paths = {source: (None, 0)}
            #The variable 'current_node' does basically store the node we are on, we initialize it with the source
            # vertex in the beginning
            current_node = source
            # The variable 'visited' is a set keeping trace of the visited nodes.
            visited = set()
            # The variable 'heap' is a Fibonacci heap we use to store our nodes and order them by
            # their current weight. To create it we resorted to an existing library which we modified
            # to better meet our requirements.
            heap = Fibonacci_heap()

            while current_node != destination:
                if current_node not in visited:
                    # Here we add our current node to the set of visited ones
                    visited.add(current_node)
                # The variable 'destinations' is essentially an adjacency list, it extracts all the edges
                # departing from the current node
                destinations = [elem[1] for elem in graph.edges(current_node)]
                # The variable 'current_node_weight' stores the weight attribute on the edge connected
                current_node_weight = shortest_paths[current_node][1]

                # During the following loop we visit all the nodes connected to the current one
                for next_node in destinations:
                    # Here we compute the weight of current edge as the sum:
                    # weight of the edge + weight of edges previously visited 
                    weight = get_weight(graph, current_node, next_node, measure) + current_node_weight
                    # Here we add nodes to the heap
                    if next_node not in visited and next_node not in heap.nodes:
                        heap.enqueue(next_node, weight)

                    # Here we add a new node to the shortest path, or update
                    # it if the current path is shorter than previous path
                    if next_node not in shortest_paths:
                        shortest_paths[next_node] = (current_node, weight)
                    else:
                        current_shortest_weight = shortest_paths[next_node][1]
                        if current_shortest_weight > weight:
                            shortest_paths[next_node] = (current_node, weight)

                # If our heap is empty, we cannot continue
                # nor reach the destination
                if not heap.__bool__():
                    return "Not Possible"
                
                # Here we update current_node with the next one,
                # namely the destination with lowest weight
                current_node = heap.dequeue_min().m_elem

            # Creating a path and reversing it
            path = []
            while current_node is not None:
                path.append(current_node)
                # Extract the previous node from shortest_paths
                next_node = shortest_paths[current_node][0]
                # Update current_node
                current_node = next_node
            # Reverse path
            path = path[::-1]
            return (path, shortest_paths[path[-1]][1])
        else:
            print('The graph is not connected.')
    else:
        print('Invalid measure, please try again.')
        
# Custom function to apply the Dijkstra algorithm in order to obtain a shortest ordered route
def shortest_ordered_route(graph, source, destinations, measure = 'network'):
    # Storing the source vertex in a variable with the same name
    source  = source
    # List variable to store the path 
    total_steps = []
    # List variable to store the chosen distance measure
    total_weight = 0
    # Iterating over the elements in the list of different destinations to reach
    for destination in destinations:
        # An in condition to avoid printing two times the same node
        if not total_steps:
            # Storing the path and the sum of weights
            steps, weight = dijkstra(graph, source, destination, measure)
            # Updating the variables 
            total_steps +=steps
            total_weight += weight
            source = destination
        # Same steps as above, with just one more: we pop the first elements in the list
        # of steps, which is the same one in the last postion of the other list
        else:
            steps, weight = dijkstra(graph, source, destination, measure)
            steps.pop(0)
            total_steps += steps
            total_weight += weight
            source = destination
    # Printing the output, conditioned to the selected measure
    print("Path:")
    print(*total_steps)
    if measure == 'network':
        print("Network distance:")
        print(total_weight)
    elif measure == 'time':
        print("Time distance")
        print(total_weight)
    elif measure == 'physical':
        print("Physical distance:")
        print(total_weight)
    return total_steps, total_weight

In [62]:
t = time.time()
s_o_r = shortest_ordered_route(G, 1, [5, 8, 25], 'time')
print(time.time() - t)

Path:
1 1803 1802 1050022 2208 2207 1050873 2866 1050871 1050346 2209 2210 1050351 1050354 1050355 2231 2235 2234 1050367 2230 2229 2237 1050369 2238 1050368 6 5 1048581 8 1050376 2246 1050304 1050305 2647 2157 1050306 1050308 2159 34 1048603 1048598 28 1050699 1050700 1048601 31 1050698 1048627 64 2655 2656 2657 1048626 179 180 35 1048604 1048606 2662 1050717 2661 40 1048609 1050708 43 1048612 2384 1048594 23 24 1048595 25
Time distance
740889
14.364387035369873


In [67]:
s_o_r[0][2]

1802

In [64]:
coordinates.set_index('ID_Node')['Latitude'][s_o_r[0]])/1000000

SyntaxError: invalid syntax (<ipython-input-64-0c3074297580>, line 1)

In [68]:
m = folium.Map(location = ((coordinates.set_index('ID_Node')['Latitude'][s_o_r[0][0]])/1000000, 
                           (coordinates.set_index('ID_Node')['Longitude'][s_o_r[0][0]])/1000000), zoom_start = 10, tiles = 'openstreetmap')
for node in s_o_r[0]:
    folium.CircleMarker(location = ((coordinates.set_index('ID_Node')['Latitude'][node])/1000000, 
                                    (coordinates.set_index('ID_Node')['Longitude'][node])/1000000), radius = 3, line_color = '#3186cc', fill_color = '#FFFFFF', fill_opacity = 0.7, fill = True).add_to(m)
    
display(m)